In [1]:
from EDTpy.ChipElements import *
import numpy as np
import copy
from EDTpy import settings
import EDTpy
from EDTpy import EmptyGeometry
from EDTpy import EmptyPath
from EDTpy.settings import *
import gdspy

# setting cpw parameters
CPW.default_values['S'] = 10
CPW.default_values['W'] = 7.1
CPW.default_values['layer'] = 0

dc_s = 2
dc_w = 2

In [ ]:
%pip install gdspy
%pip install tqdm

In [2]:
class Marker(EmptyGeometry):
    default_values = {
        "a": 20,
        "b": 100,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "Marker"
        
        a = values['a']
        b = values['b']
        layer = values['layer']
        
        self + gdspy.Rectangle([-b/2-b/10, -b/2-b/10],
                               [b/2+b/10, b/2+b/10], layer = 0)
        
        mark = gdspy.boolean(gdspy.Rectangle([-b/2, -a/2],[b/2, a/2]), gdspy.Rectangle([-a/2, -b/2],
                               [a/2, b/2]), 'or', layer = 0)
        self - mark
        self.add_port([0, 0], 0)

In [3]:
class Planar_inductor(EmptyGeometry):
    default_values = {
        "a": dc_s,
        "b": dc_w,
        "gap": dc_w,
        "N_sp": 12,
        "width": 200,
        "leg": 0,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "Planar_inductor"
        
        a = values['a']
        b = values['b']
        N_sp = values['N_sp']
        w = values['width']
        gap = values['gap']
        leg = values['leg']
        layer = values['layer']

        self + gdspy.Rectangle([0, w+b],
                               [(a+b)*2*(N_sp)+4*b+a+b, -w-b], layer = 0)

        self - gdspy.Rectangle([0, -a/2],
                               [2*b+b+a, a/2], layer = 0)
        
        self - gdspy.Rectangle([2*b+b, a/2],
                               [2*b+b+a, w+a/2-gap], layer = 0)

        for i in range(N_sp):
            self - gdspy.Rectangle([2*b+b+a+(2*a+2*b)*(i), w+a/2-gap],
                                   [2*b+b+a+b+(2*a+2*b)*(i), w-gap-a/2], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+(2*a+2*b)*(i), w-gap+a/2],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i), -w-a/2+gap], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+a+(2*a+2*b)*(i), -w-a/2+gap],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i)+b, -w-a/2+gap+a], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+a+b+(2*a+2*b)*(i), w-gap+a/2],
                                   [2*b+b+a+b+a+a+b+(2*a+2*b)*(i), -w-a/2+gap], layer = 0)

        self + gdspy.Rectangle([2*b+b+(2*a+2*b)*(N_sp), a/2],
                               [2*b+b+a+(2*a+2*b)*(N_sp), w+a/2-gap], layer = 0)
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+a+b+b*2, -a/2],
                               [(a+b)*2*(N_sp)+a+b+b*4, a/2], layer = 0)  
        #leg for connection
        x_start = (a+b)*2*(N_sp)+a+b+b*4
        self + gdspy.Rectangle([x_start, -CPW.default_values['S']/2-CPW.default_values['W']],
                               [x_start+leg, CPW.default_values['S']/2+CPW.default_values['W']], layer = 0)  
        self - gdspy.Rectangle([x_start, -CPW.default_values['S']/2],
                               [x_start+leg, CPW.default_values['S']/2], layer = 0)  

        self.add_port([x_start+leg, 0], 0)
        self.add_port([0, 0], 180)

In [4]:
class Planar_capasitor(EmptyGeometry):
    default_values = {
        "a": dc_s,
        "b": dc_w,
        "gap": dc_w,
        "N_sp": 15,
        "width": 120,
        "layer": 0,
        "leg": 0
    }

    def _drawing(self, values):
        self.name = "Planar_capasitor"
        
        a = values['a']
        b = values['b']
        N_sp = values['N_sp']
        w = values['width']
        gap = values['gap']
        layer = values['layer']
        leg = values['leg']

        self + gdspy.Rectangle([0, w],
                               [(a+b)*2*(N_sp)+4*b+a+b, -w], layer = 0)

        self - gdspy.Rectangle([0, -a/2],
                               [2*b, a/2], layer = 0)

        for i in range(N_sp):

            self - gdspy.Rectangle([2*b+(2*a+2*b)*(i), -a/2],
                                   [2*b+(2*a+2*b)*(i+1), a/2], layer = 0)
            
            self - gdspy.Rectangle([2*b+b+(2*a+2*b)*(i), w],
                                   [2*b+b+a+(2*a+2*b)*(i), a/2+gap], layer = 0)
            self - gdspy.Rectangle([2*b+b+(2*a+2*b)*(i), -w],
                                   [2*b+b+a+(2*a+2*b)*(i), -a/2-gap], layer = 0)
            
            self - gdspy.Rectangle([2*b+b+a+b+(2*a+2*b)*(i), w-gap],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i), a/2], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+(2*a+2*b)*(i), -w+gap],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i), -a/2], layer = 0)
            
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+b*2, -a/2],
                               [(a+b)*2*(N_sp)+a+b+2*b, a/2], layer = 0)
                    
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+b*2+a, w],
                               [(a+b)*2*(N_sp)+b*2+a+b, a/2+gap], layer = 0)
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+b*2+a, -w],
                               [(a+b)*2*(N_sp)+b*2+a+b, -a/2-gap], layer = 0)
        
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+a+b+b*2, -a/2],
                               [(a+b)*2*(N_sp)+a+b+b*4, a/2], layer = 0)  
        
        x_start = (a+b)*2*(N_sp)+a+b+b*4
        self + gdspy.Rectangle([x_start, -CPW.default_values['S']/2-CPW.default_values['W']],
                               [x_start+leg, CPW.default_values['S']/2+CPW.default_values['W']], layer = 0)  
        self - gdspy.Rectangle([x_start, -CPW.default_values['S']/2],
                               [x_start+leg, CPW.default_values['S']/2], layer = 0)

        self.add_port([x_start+leg, 0], 0)
        self.add_port([0, 0], 180)

In [5]:
class Contact_pad_spiral(EmptyGeometry):
    default_values = {
        "a": dc_s,
        "b": dc_w,
        "N_sp": 6,
        "contact_pad_size": 300,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "Contact_pad_spiral"
        
        a = values['a']
        b = values['b']
        N_sp = values['N_sp']
        contact_pad_size = values['contact_pad_size']
        layer = values['layer']

        self + gdspy.Rectangle([contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3, (contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3)],
                                 [-(contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3), -(contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3)], layer = 0)

        
        self - gdspy.Rectangle([-contact_pad_size/2, -contact_pad_size/2],
                                 [contact_pad_size/2, contact_pad_size/2], layer = 0)

        self - gdspy.Rectangle([-a/2, contact_pad_size/2],
                                 [a/2, contact_pad_size/2+b], layer = 0)
        self - gdspy.Rectangle([-a/2, contact_pad_size/2+b],
                                 [contact_pad_size/2+b, contact_pad_size/2+b+a], layer = 0)

        for i in range(N_sp):

            self - gdspy.Rectangle([contact_pad_size/2+b+(a+b)*(i), contact_pad_size/2+b+a+(a+b)*(i)],
                                     [contact_pad_size/2+b+a+(a+b)*(i), -contact_pad_size/2-b-(a+b)*(i)], layer = 0)

            self - gdspy.Rectangle([-contact_pad_size/2-b-(a+b)*(i), -contact_pad_size/2-b-a-(a+b)*(i)],
                                     [contact_pad_size/2+b+a+(a+b)*(i), -contact_pad_size/2-b-(a+b)*(i)], layer = 0)

            self - gdspy.Rectangle([-contact_pad_size/2-b-(a+b)*(i), -contact_pad_size/2-b-a-(a+b)*(i)],
                                     [-contact_pad_size/2-b-a-(a+b)*(i), +contact_pad_size/2+b+(a+b)*(i+1)], layer = 0)

            self - gdspy.Rectangle([-contact_pad_size/2-b-a-(a+b)*(i), contact_pad_size/2+b+(a+b)*(i+1)],
                                     [contact_pad_size/2+b+(a+b)*(i+1), contact_pad_size/2+b+a+(a+b)*(i+1)], layer = 0)

        self - gdspy.Rectangle([contact_pad_size/2+b+(a+b)*(N_sp), contact_pad_size/2+b+a+(a+b)*(N_sp)],
                                 [contact_pad_size/2+b+a+(a+b)*(N_sp), a/2], layer = 0)  

        self - gdspy.Rectangle([contact_pad_size/2+b+(a+b)*(N_sp), a/2],
                                 [contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3, -a/2], layer = 0)  

        self.add_port([contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3, 0], 0)
        self.add_port([0, 0], 180)

In [6]:
class LCLC(EmptyGeometry):
    default_values = {
        "a": 20,
        "b": 100,
        "layer": 0,
        "w": 270,
        "N_sp": 10,
        "N_c1": 10,
        "N_ind": 10,
        "N_c2": 10,
        "line": 2,
    }

    def _drawing(self, values):
        self.name = "Marker"
        
        a = values['a']
        b = values['b']
        layer = values['layer']
        w=values['w']

        N_sp=values['N_sp']
        N_c1=values['N_c1']
        N_ind=values['N_ind']
        N_c2=values['N_c2']
        
        line=values['line']
        
        spiral = Contact_pad_spiral(N_sp=N_sp)
#         spiral.merge_with(self.ports[9], 1)
        self + spiral
        cap = Planar_capasitor(width=w, N_sp=N_c1, leg = 0)
        cap.merge_with(spiral.ports[0], 0)
        self + cap
        ind_oo = Planar_inductor(width=w, N_sp=N_ind, leg = 0)
        ind_oo.merge_with(cap.ports[1], 0)
        self + ind_oo
        cap_o1o = Planar_capasitor(width=w, N_sp = N_c2, leg = 0)
        cap_o1o.merge_with(ind_oo.ports[1], 0)
        self + cap_o1o
        x = cap_o1o.ports[1].position[0]
        y = cap_o1o.ports[1].position[1]
        self.add_port([0, 0], 0)
        self.add_port([x, y], 0)
        

In [7]:
class GraphenHole(EmptyGeometry):
    default_values = {
        "size": 200,
        "layer": 0,
        "contact_S": 2,
        "contact_b": 12,
        "contact_L": 30,        
    }

    def _drawing(self, values):
        self.name = "GraphenHole"
        
        self + gdspy.Rectangle([-values['size']/2, -values['size']/2],
                               [values['size']/2, values['size']/2],
                               layer = values['layer'])
        
        #Markers
        mark = gdspy.boolean(gdspy.Rectangle([-55, 50.6],[-45, 49.4]), gdspy.Rectangle([-50.6, 55],
                               [-49.4, 45]), 'or', layer = 0)
       
        for i in range(4):
            mark.rotate(np.pi/2)
            self - mark
            
        leg_central = gdspy.Rectangle([-values['size']/2, -CPW.default_values['S']/2] ,
                               [-values['size']/2 + values['contact_L'], CPW.default_values['S']/2],
                               layer = values['layer'])
        self - leg_central
        
        leg_GND = gdspy.Rectangle([-values['size']/2, -CPW.default_values['S']/2+values['size']/6] ,
                               [-values['size']/2 + values['contact_L'], CPW.default_values['S']/2+values['size']/6],
                               layer = values['layer'])
        self - leg_GND.rotate(180/180*np.pi)
        leg_central = gdspy.Rectangle([-values['size']/2, -values['contact_S']/2] ,
                               [-values['size']/2 + values['contact_L'], values['contact_S']/2],
                               layer = values['layer'])
        re_central = gdspy.Rectangle(
                               [-values['size']/2 + values['contact_L']+10, -values['contact_S']/2-5] ,
                               [-values['size']/2 + values['contact_L'], values['contact_S']/2+5],
                               layer = values['layer'])
        
        
        leg_corner_S = gdspy.Rectangle([-values['size']/2*np.sqrt(2), -values['contact_S']/2],
                               [-values['size']/2*np.sqrt(2) + values['contact_L']*np.sqrt(2), values['contact_S']/2],
                               layer = values['layer'])
        re_corner_S = gdspy.Rectangle(
                           [-values['size']/2*np.sqrt(2) + values['contact_L']*np.sqrt(2)+10, -values['contact_S']/2-5],
                           [-values['size']/2*np.sqrt(2) + values['contact_L']*np.sqrt(2), values['contact_S']/2+5],
                           layer = values['layer'])
        
        leg_corner_W = gdspy.Rectangle([-values['size']/2*np.sqrt(2), -values['contact_b']/2],
                               [-values['size']/2*np.sqrt(2) + values['contact_L']*np.sqrt(2), values['contact_b']/2],
                               layer = values['layer'])
        
        leg_corner_S.rotate(-135/180*np.pi)
        re_corner_S.rotate(-135/180*np.pi)
        leg_corner_W.rotate(-135/180*np.pi)
        
        for i in range(4):
            self - leg_central.rotate(90/180*np.pi*i)
#             self - re_central.rotate(90/180*np.pi*i)
            
            self + leg_corner_W.rotate(90/180*np.pi*i)
            self - leg_corner_S.rotate(90/180*np.pi*i)
            self - re_corner_S.rotate(90/180*np.pi*i)
            
            
        leg_gate = gdspy.Rectangle([-values['size']/2, -values['contact_S']/2+values['size']/5] ,
                               [-values['size']/2 + values['contact_L'], values['contact_S']/2+values['size']/5],
                               layer = values['layer'])
        leg_gate_b = gdspy.Rectangle([-values['size']/2, values['contact_S']/2-values['size']/5] ,
                       [-values['size']/2 + values['contact_L'], -values['contact_S']/2-values['size']/5],
                       layer = values['layer'])
        re_gate = gdspy.Rectangle(
                               [-values['size']/2 + values['contact_L']+10, -values['contact_S']/2+values['size']/5-5] ,
                               [-values['size']/2 + values['contact_L'], values['contact_S']/2+values['size']/5+5],
                               layer = values['layer'])
        

        port_central = EDTpy.Port((-values['size']/2, 0), 180)
        port_corner = EDTpy.Port((-values['size']/2, values['size']/2), 180-45)
#         self.ports = [port_central.rotate(90, center = (0, 0)) for i in range(4)]
#         self.ports += [port_corner.rotate(90, center = (0, 0)) for i in range(4)]
        for i in range(4):
            self.ports += [copy.deepcopy(port_central)]
            self.ports += [copy.deepcopy(port_corner)]
            port_central.rotate(90, center = (0, 0))
            port_corner.rotate(90, center = (0, 0))
            

In [8]:
# sketch = EDTpy.EmptySketch()
# test = GraphenHole()
# sketch.add_geometry(test)
# test.show()

In [9]:
class CPW_C(EmptyGeometry):
    default_values = {
        "s": CPW.default_values['S'],
        "w": CPW.default_values['W'],
        "N": 2, #number of odd elements
        "l": 50, 
        "a": 3.3, # width of finger
        "b": 3.3, # gap between fingers
        "g": 15, # gap between fingers and GND
        "contact_pad_size": 200,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "CPW_C"
        
        w = values['w']
        s = values['s']
        N = int((values['N'])/2)
        a = values['a']
        b = values['b']
        l = values['l']+b*2
        g = values['g']
        gnd = 30
        
        contact_pad_size = values['contact_pad_size']
        layer = values['layer']
        
        self + gdspy.Rectangle([0, -s/2-w],
                               [10, s/2+w], layer = 0)
        self - gdspy.Rectangle([0, -s/2],
                               [10, s/2], layer = 0)
        
        L = l+gnd
        jo = (N)*(a+b)*2+a/2
        jo_g = jo+g
        offset = L*0.4
        points = [(10, s/2+w), 
                  (offset, jo_g), (offset+L, jo_g), 
                  (offset+L, jo), (offset, jo), (10, s/2)]
        self + gdspy.Polygon(points)
        points = [(10, -s/2-w), 
                  (offset, -jo_g), (offset+L, -jo_g), 
                  (offset+L, -jo), (offset, -jo), (10, -s/2)]
        self + gdspy.Polygon(points)
        
        offset1 = offset*1.3
        self + gdspy.Rectangle([offset1, jo],
                               [offset1+l, -jo], layer = 0)
        
        self - gdspy.Rectangle([offset1,      a/2],
                               [offset1+l-b*4,  -a/2], layer = 0)
        
        for n in range(N):
            self - gdspy.Rectangle([offset1,     a/2+2*(a+b)*(n+1)-b],
                                   [offset1+l-b*4, a/2+2*(a+b)*(n+1)-b+a], layer = 0)
            self - gdspy.Rectangle([offset1,     -a/2-2*(a+b)*(n+1)+b],
                                   [offset1+l-b*4, -a/2-2*(a+b)*(n+1)+b-a], layer = 0)
            
            self - gdspy.Rectangle([offset1+b*4,     a/2+2*(a+b)*(n)+b],
                                   [offset1+l,     a/2+2*(a+b)*(n)+a+b], layer = 0)
            self - gdspy.Rectangle([offset1+b*4,    -a/2-2*(a+b)*(n)-b],
                                   [offset1+l,    -a/2-2*(a+b)*(n)-a-b], layer = 0)
        
        z = offset+L
        self + gdspy.Rectangle([z+100, -contact_pad_size/2-50],
                               [z+100+contact_pad_size+50, contact_pad_size/2+50], layer = 0)
        self - gdspy.Rectangle([z+100, -contact_pad_size/2],
                       [z+100+contact_pad_size, contact_pad_size/2], layer = 0)
        
        points = [(z, jo_g), 
                  (z+100, contact_pad_size/2+50), (z+100, contact_pad_size/2), 
                  (z, jo)]
        self + gdspy.Polygon(points)
        
        points = [(z, -jo_g), 
                  (z+100, -contact_pad_size/2-50), (z+100, -contact_pad_size/2), 
                  (z, -jo)]
        self + gdspy.Polygon(points)

        self.add_port([0, 0], 180)

In [10]:
sketch = EDTpy.EmptySketch()
sketch + gdspy.Rectangle([-5000/2, -5000/2],
                         [5000/2, 5000/2], layer = 0)

sketch - gdspy.Rectangle([-5000/2 + 250, -5000/2 + 250],
                         [5000/2 - 250, 5000/2 - 250], layer = 0)

sketch.add_port([-2000, 2000], -180) #Marker port  --  0

CPW.default_values['S'] = 10
CPW.default_values['W'] = 7.1
marker = Marker()
marker.merge_with(sketch.ports[0], 0)
sketch.add_geometry(marker)
sketch.circular_array(marker)

sketch.add_port([-1500, 0], 180) #Marker port  --  1

res_path = [[0, 0], [300, 0], [300, -600], [300, -1500], [800, -1500], [800, 0], [1300, 0]]
res_R = 200
res = CPW(res_path, res_R)
print(res.length)
CPW_c_r = CPW_C()
CPW_c_r.merge_with(sketch.ports[1], 0)
sketch.add_geometry(CPW_c_r)
res.merge_with(CPW_c_r.ports[0], 0)
sketch.add_geometry(res)

G = GraphenHole()
G.merge_with(res.ports[1], 0)
sketch.add_geometry(G)

x0 = -1800
y0 = 1300
text = gdspy.Text("W1", 250, (x0-145, y0+33))
sketch + text 


CPW.default_values['S'] = 2
CPW.default_values['W'] = 2

sketch.add_port([1800, -1500], 0) #DC port  -- 2
lclc = LCLC(N_sp=25, N_c1=40, N_ind=40, N_c2=50, line=CPW.default_values['S'])
lclc.merge_with(sketch.ports[2], 0)
sketch.add_geometry(lclc)

res_path = [G.ports[3].position, [G.ports[3].position[0]-100, G.ports[3].position[0]], 
            [lclc.ports[1].position[0]-350, lclc.ports[1].position[1]],
            [lclc.ports[1].position[0]-150, lclc.ports[1].position[1]], lclc.ports[1].position]
res_R = 200
res = CPW(res_path, res_R)
sketch.add_geometry(res)


sketch.add_port([1800, -800], 0) #DC port  -- 3
lclc = LCLC(N_sp=25, N_c1=40, N_ind=40, N_c2=50, line=CPW.default_values['S'])
lclc.merge_with(sketch.ports[3], 0)
sketch.add_geometry(lclc)

res_path = [G.ports[2].position, [G.ports[2].position[0], G.ports[2].position[0]-300], 
            [lclc.ports[1].position[0]-150, lclc.ports[1].position[1]],
            [lclc.ports[1].position[0]-50, lclc.ports[1].position[1]], lclc.ports[1].position]
res_R = 200
res = CPW(res_path, res_R)
sketch.add_geometry(res)


sketch.add_port([1800, -100], 0) #DC port  -- 4
lclc = LCLC(N_sp=25, N_c1=40, N_ind=40, N_c2=50, line=CPW.default_values['S'])
lclc.merge_with(sketch.ports[4], 0)
sketch.add_geometry(lclc)

res_path = [G.ports[5].position, [G.ports[5].position[0]+100, G.ports[5].position[0]-200], 
            [lclc.ports[1].position[0]-100, lclc.ports[1].position[1]], lclc.ports[1].position]
res_R = 200
res = CPW(res_path, res_R)
sketch.add_geometry(res)

sketch.add_port([1800, 600], 0) #DC port  -- 5
lclc = LCLC(N_sp=25, N_c1=40, N_ind=40, N_c2=50, line=CPW.default_values['S'])
lclc.merge_with(sketch.ports[5], 0)
sketch.add_geometry(lclc)

res_path = [G.ports[4].position, [G.ports[4].position[0]+200, G.ports[4].position[0]], 
            [lclc.ports[1].position[0]-150, lclc.ports[1].position[1]],
            [lclc.ports[1].position[0]-50, lclc.ports[1].position[1]], lclc.ports[1].position]

res_R = 150
res = CPW(res_path, res_R)
sketch.add_geometry(res)


sketch.add_port([1800, 1300], 0) #DC port  -- 6
lclc = LCLC(N_sp=25, N_c1=40, N_ind=40, N_c2=50, line=CPW.default_values['S'])
lclc.merge_with(sketch.ports[6], 0)
sketch.add_geometry(lclc)

res_path = [G.ports[7].position, [G.ports[7].position[0]+100, G.ports[7].position[0]+200], 
            [lclc.ports[1].position[0]-150, lclc.ports[1].position[1]],
            [lclc.ports[1].position[0]-50, lclc.ports[1].position[1]], lclc.ports[1].position]
res_R = 150
res = CPW(res_path, res_R)
sketch.add_geometry(res)

sketch.add_port([-300, 1800], 90) #DC port  -- 7
lclc = LCLC(N_sp=25, N_c1=40, N_ind=40, N_c2=50, line=CPW.default_values['S'])
lclc.merge_with(sketch.ports[7], 0)
sketch.add_geometry(lclc)

res_path = [G.ports[6].position, [G.ports[6].position[0], G.ports[6].position[0]+300], 
            [lclc.ports[1].position[0], lclc.ports[1].position[1]-100], lclc.ports[1].position]
res_R = 150
res = CPW(res_path, res_R)
sketch.add_geometry(res)

sketch.add_port([-1000, 1800], 90) #DC port  -- 8
lclc = LCLC(N_sp=25, N_c1=40, N_ind=40, N_c2=50, line=CPW.default_values['S'])
lclc.merge_with(sketch.ports[8], 0)
sketch.add_geometry(lclc)

res_path = [G.ports[1].position, [G.ports[1].position[0]-100, G.ports[1].position[0]+400], 
            [lclc.ports[1].position[0], lclc.ports[1].position[1]-200], lclc.ports[1].position]
res_R = 200
res = CPW(res_path, res_R)
sketch.add_geometry(res)

sketch.assemble()
sketch.save_gds("/Applications/W1")
sketch.show()

3956.637061435917


Assembling Empty Sketch: 100%|██████████| 21/21 [00:00<00:00, 54.52it/s] 
/Applications/Gdspy/EDTpy/geometry.py:222: UserWarning: [GDSPY] Polygons with more than 8190 are not supported by the official GDSII specification.  This GDSII file might not be compatible with all readers.
  gdspy.current_library.write_gds(filename + '.gds')
2025-01-24 07:19:47.170 Python[4030:141431] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-24 07:19:47.170 Python[4030:141431] +[IMKInputSession subclass]: chose IMKInputSession_Modern


"'Empty Sketch', (0.0, 0.0), 0.0 deg, 9 ports\n"